# Week of July 1, 2019
## Review
### Ordered Dict
- LRUCache
    - the solution class can be made OrderedDict

### Coding style
- Integer to English words
    - divide the words in to groups of < 10, < 20, < 100, < 1000
    - always keep ' ' at the end of a word
    - use str.strip() to remove ' ' at the end


### Recursion/Stack
- number of islands

### binary search
- median of two sorted array
    - split the arrays so that left and right have equal size, then try to find the median using binary search

### Heap
- meeting room II (alternative solution exists)
    - maintain a heap of end time, where each entry is a room
    - add float('inf') and float('-inf') to the end of the input

### Greedy
- meeting room II
    - sort start and end time first
    - use another variable to keep number of availabla rooms
- Find the celebrity
    - Find the candidate for celebrity first, then check this is actually the celebrity
- Best Time to Buy and Sell Stock II
    - sum all the positive increases
- Queue Reconstruction by Height
    - first sort descending by height h, then within each height group sort ascending by k
    - insert each p in people into res at position k in order of the sorted input 
- Gas Station
    - If you can't pass location i from any previous locations, you can't pass location i starting at location i
- Partition Labels
    - for each letter, first compute intervals [start, end] where start and end are the first and last indices where the letter occurs
    - then the idea is: merge overlapping intervals and count the size of each merged interval
- Remove Duplicate Letters
    - RDL2:
    - keep a stack for ans, a dic for how many of each letters are there
    - at every stage, if something is already in ans, skip this iteration and count -= 1
    - if you see a new letter, remove from ans as many letters as than are > current letter and you will see the removed letters in the future (i.e. count > 0)
    - add the current letter to ans
    
    
### Notes
- from collections import OrderedDict
- (See Median of Two Sorted Array). If you need to match a few criterions, maybe try to match another specific one a priori, and try to the match the rest later. i.e. you need to make sure A and B hold. You can easily get A to hold or B to hold, but not both. Don't just focus on the case where you make A hold and try to obtain B. Maybe it is easier to get B to hold first, then to obtain A.
- it is useful to augment your input say by float('inf'), float('-inf') to avoid boundary cases.

### July 3, 2019
- LRUCache
    - LRUCache is my solution
    - LRUCache2 using a built-in ordered dict is much faster
- Number of Islands
- Median of Two Sorted Array
    - MTSA: very fast. 
    - just have to be very careful about the indices
    - you have two choice: 
        - split the array so that the two parts have equal size or 
        - split the array such that the place where you split is the median of one of the array and try
    - the first way is the better way
    - it is useful to augment your input say by float('inf'), float('-inf') to avoid boundary cases.
- Meeting Rooms II
    - MR1: my way, pretty fast using heapq
    - MR2: alternative solution using the essence of the problem
- Find the Celebrity
    - This is not hard. This is a typical coding style problem. FC1 is very fast.

In [212]:
from collections import deque
class Node:
    def __init__(self, key):
        self.key = key
        self.prev = None
        self.next = None
    
    def printList(self):
        temp = self
        ans = []
        while temp != None:
            ans.append(temp.key)
            temp = temp.next
        print(ans)
        
class LRUCache:
    def __init__(self, capacity: int):
        self.dic = {}
        self.cap = capacity
        self.cur_cap = 0
        self.head = None
        self.tail = None

    def get(self, key: int) -> int:
        if key in self.dic:
            x = self.dic[key][1]
            #x.prev.printList()
            if x.prev and x.next:
                #print('prev')
                x.prev.next = x.next
                x.next.prev = x.prev
                
                self.tail.next = x
                x.prev = self.tail
                x.next = None
                self.tail = x
            elif x.next:
                #print('next')
                
                self.head = x.next
                self.head.prev = None
                

                self.tail.next = x
                x.prev = self.tail
                self.tail.next.next = None
                self.tail = x
            return self.dic[key][0]
        else:
            return -1
        
    def put(self, key: int, value: int) -> None:
        if self.cur_cap < self.cap and key not in self.dic:
            x = Node(key)
            self.dic[key] = [value, x]
            
            if self.head:
                self.tail.next = x
                x.prev = self.tail
                self.tail = x
            else:
                self.head = x
                self.tail = x
                
            self.cur_cap += 1
        elif key in self.dic:
            self.dic[key][0] = value
            self.get(key)           
        else:
            old_key = self.head.key
            self.head = self.head.next
            if self.head:
                self.head.prev = None
            del self.dic[old_key]
            self.cur_cap -= 1
            self.put(key, value)
            
            
        

In [213]:
def xput(a,b, x):
    x.put(a,b)
    #print(x.dic)
    if x.head:
        print('x.input', a)
        x.head.printList()
        x.tail.printList()

def xget(a, x):
    print('x.get:', a, x.get(a))
    if x.head:
        x.head.printList()
        x.tail.printList()

def test(cap, lst):
    x = LRUCache(cap)
    for cmd in lst:
        if len(cmd) == 1:
            xget(cmd[0], x)
        else:
            xput(cmd[0], cmd[1], x)

In [214]:
test(2,[[2,1],[2],[3,2],[2],[3]])

x.input 2
[2]
[2]
x.get: 2 1
[2]
[2]
x.input 3
[2, 3]
[3]
x.get: 2 1
[3, 2]
[2]
x.get: 3 2
[2, 3]
[3]


In [ ]:
class LRUCache2(OrderedDict):
    def __init__(self, capacity: int):
        self.cap = capacity


    def get(self, key: int) -> int:
        if key not in self:
            return -1
        
        self.move_to_end(key)
        return self[key]
        
    def put(self, key: int, value: int) -> None:
        if key in self:
            self.move_to_end(key)
        self[key] = value
        if len(self) > self.cap:
            self.popitem(last = False)

In [289]:
class MTSA:
    def findMedianSortedArrays(self, nums1, nums2) -> float:
        n = len(nums1)
        m = len(nums2)
        
        if n == 0 and m == 0:
            return 0
        elif n == 0:
            return nums2[m//2] if m % 2 == 1 else (nums2[m//2] + nums2[m//2 - 1])/2
        elif m == 0:
            return nums1[n//2] if n % 2 == 1 else (nums1[n//2] + nums1[n//2 - 1])/2
        
        nums1.append(float('inf'))
        nums1.append(float('-inf'))
        nums2.append(float('inf'))
        nums2.append(float('-inf'))
        
        if n > m:
            nums1, nums2 = nums2, nums1
            n, m = m, n
        
        #print('nums1', nums1)
        #print('nums2', nums2)
        
        left = 0
        right = n+1
        while left < right:
            i = (left + right) // 2
            j = (n+m) // 2 - i
            #print(i,j)
            #print(nums1[i], nums2[j-1], nums2[j], nums1[i-1])
            if nums1[i] >= nums2[j-1] and nums2[j] >= nums1[i-1]:
                if (n+m) % 2 == 1:
                    return min(nums1[i], nums2[j])
                else:
                    return (min(nums1[i], nums2[j]) + max(nums1[i-1], nums2[j-1])) / 2
            elif nums1[i] < nums2[j-1]:
                left = i + 1
            else:
                right = i
        
        #print('out of the while loop')
        #print(i,j)
        #print(nums1[i], nums2[j-1], nums2[j], nums1[i-1])
        return (min(nums1[i], nums2[j]) + max(nums1[i-1], nums2[j-1])) / 2
    
x = MTSA()

In [290]:
print(x.findMedianSortedArrays([2], [1,9,10]))

5.5


In [2]:
import heapq
class MR1:
    def minMeetingRooms(self, intervals) -> int:
        if not intervals:
            return 0
        
        heapq.heapify(intervals)
        res = []
        
        
        while intervals:
            interval = heapq.heappop(intervals)
            if not res or res[0] > interval[0]:
                heapq.heappush(res, interval[1])
            else:
                heapq.heappushpop(res, interval[1])
        
        return len(res)

In [3]:
class MR2:
    def minMeetingRooms(self, intervals) -> int:
        start = []
        end = []
        for i in intervals:
            start.append(i[0])
            end.append(i[1])
        start.sort()
        end.sort()
        s = 0
        e = 0
        rooms = 0
        available = 0
        while s < len(start):
            if start[s] < end[e]:
                if available:
                    available -= 1
                else:
                    rooms += 1
                s += 1
            else:
                available += 1
                e += 1
        return rooms

In [4]:
class FC1:
    def findCelebrity(self, n):
        celebrity = 0
        for i in range(1, n):
            if knows(celebrity, i):
                celebrity = i
        
        for i in range(celebrity):
            if knows(celebrity, i):
                return -1
            
        if i in range(n):
            if not knows(i, celebrity):
                return -1
        
        return celebrity

### July 4, 2019
- Integer to English Words

In [6]:
class IEW(object):
    def numberToWords(self, num):
        if num == 0:
            return 'Zero'
        
        self.dic = {
                0:'',
                1:'One', 2:'Two', 3:'Three', 4:'Four', 5:'Five', 6:'Six', 7:'Seven',
                8:'Eight', 9:'Nine', 10:'Ten', 11:'Eleven', 12:'Twelve', 13:'Thirteen',
                14:'Fourteen', 15:'Fifteen', 16:'Sixteen', 17:'Seventeen',
                18:'Eighteen', 19:'Nineteen', 20:'Twenty', 30:'Thirty',
                40:'Forty', 50:'Fifty', 60:'Sixty', 70:'Seventy', 80:'Eighty',
                90:'Ninety'
              }
        x = [' Billion ', ' Million ', ' Thousand ', '']
        
        res = ''
        while num:
            ThreeDigits = num % 1000
            if num // 1000:
                res = self.helper(ThreeDigits) + x.pop() + res
            else:
                x.pop()
                res = self.helper(ThreeDigits) + res
            num //= 1000
        
        if res[-1] == ' ':
            return res[:-1]
        return res
        
        
    def helper(self, n):
        if n in self.dic:
            return self.dic[n]

        res = ''
        c = n // 100
        if c > 0:
            res = self.dic[c] + ' Hundred '
        
        ds = n % 100
        if ds in self.dic:  
            res += self.dic[ds]
        else:
            d = (ds // 10) * 10
            o = ds % 10
            print(ds, d, o)
            
            res += self.dic[d] + ' ' + self.dic[o]
        
        if res[-1] == ' ':
            return res[:-1]
        return res

## July 3, 2019
- Integer to English Words
    - ITEW1: me
    - ITEW2: pro. Its faster to use a helper that returns words by if its less than 10, 100, 1000, etc.

In [59]:
class ITEW1(object):
    def __init__(self):
        self.dic = {
                0:'',
                1:'One', 2:'Two', 3:'Three', 4:'Four', 5:'Five', 6:'Six', 7:'Seven',
                8:'Eight', 9:'Nine', 10:'Ten', 11:'Eleven', 12:'Twelve', 13:'Thirteen',
                14:'Fourteen', 15:'Fifteen', 16:'Sixteen', 17:'Seventeen',
                18:'Eighteen', 19:'Nineteen', 20:'Twenty', 30:'Thirty',
                40:'Forty', 50:'Fifty', 60:'Sixty', 70:'Seventy', 80:'Eighty',
                90:'Ninety'
              }
        
    def numberToWords(self, num):
        if num == 0:
            return 'Zero'
        
        
        x = [' Billion', ' Million', ' Thousand', '']
        ThreeDigits = []
        
        while num:
            temp = self.helper(num % 1000)
            if temp:
                temp += x.pop()
                ThreeDigits.append(temp)
            else:
                x.pop()
            num //= 1000
        
        print(ThreeDigits)
        return ' '.join(ThreeDigits[::-1])
        
        
    def helper(self, n):
        if n in self.dic:
            return self.dic[n]

        res = ''
        c = n // 100
        if c > 0:
            res = self.dic[c] + ' Hundred'
        
        ds = n % 100
        if ds and c:
            res += ' '
            
        if ds in self.dic:  
            res += self.dic[ds]
        else:        
            res += self.dic[(ds // 10) * 10] + ' ' + self.dic[ds % 10]
        
        return res
        
x = ITEW()        

In [58]:
class ITEW2:
    def __init__(self):
        self.lessThan20 = ['', 'One', 'Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Eleven', 'Twelve', 'Thirteen', 'Fourteen', 'Fifteen', 'Sixteen', 'Seventeen', 'Eighteen', 'Nineteen']
        self.tens = ['', 'Ten', 'Twenty', 'Thirty', 'Forty', 'Fifty', 'Sixty', 'Seventy', 'Eighty', 'Ninety']
        self.thousands = ['', 'Thousand', 'Million', 'Billion']
        
    def numberToWords(self, num):
        """
        :type num: int
        :rtype: str
        """
        if num == 0:
            return 'Zero'
        
        res = ''
        for thousand in self.thousands:
            if num % 1000 != 0:
                res = self.helper(num % 1000) + thousand + ' ' + res
            num /= 1000
        
        return res.strip()
    
    def helper(self, num):
        if num == 0:
            return ''
        elif num < 20:
            return self.lessThan20[num] + ' '
        elif num < 100:
            return self.tens[num / 10] + ' ' + self.helper(num % 10)
        elif num < 1000:
            return self.lessThan20[num / 100] + ' Hundred ' + self.helper(num % 100)

## July 4, 2019
### Greedy
- Best Time to Buy and Sell Stock II
    - sum all the positive increases
- Queue Reconstruction by Height
    - first sort descending by height h, then within each height group sort ascending by k
    - insert each p in people into res at position k

In [64]:
class QRH(object):
    def reconstructQueue(self, people):
        people.sort(key=lambda x: (-x[0], x[1]))
        
        #print(people)
        res = []
        for p in people:
            res.insert(p[1], p)
        
        return res

x = QRH()

In [65]:
x.reconstructQueue([[7,0], [4,4], [7,1], [5,0], [6,1], [5,2]])

[[5, 0], [7, 0], [5, 2], [6, 1], [4, 4], [7, 1]]

## July 5, 2019
### Greedy
- Gas Station
- Partition Labels
    - for each letter, first compute intervals [start, end] where start and end are the first and last indices where the letter occurs
    - then the idea is: merge overlapping intervals and count the size of each merged interval
- Remove Duplicate Letters
    - RDL1: my solution, slow.
    - RDL2: pro's solution, fast

In [46]:
from collections import defaultdict
class RDL1(object):
    def removeDuplicateLetters(self, s):
        n = len(s)
        dic = defaultdict(int)
        for c in s:
            dic[c] += 1
        res = []
        
        def helper(remain_ind, beg):
            j = remain_ind - 1
            
            while j > beg:
                if s[j] <= s[remain_ind] and dic[s[j]] > -1:
                    remain_ind = j
                j -= 1
            
            res.append(s[remain_ind])
            dic[s[remain_ind]] = -1
            
            return remain_ind

        beg = -1
        i = 0
        while i < n:
            dic[s[i]] -= 1
            if dic[s[i]] == 0:
                beg = helper(i, beg)
                if beg == i:
                    i += 1
                else:
                    dic[s[i]] += 1
            else:
                i += 1
        
        return ''.join(res)

x = RD1()

In [50]:
class RDL2(object):
    def removeDuplicateLetters(self, s):
        count = defultdict(int)
        for letter in s:
            count[letter] += 1
        
        stack = []
        
        for letter in s:
            if letter in stack:
                count[letter] -= 1
                continue
            while stack and letter < stack[-1] and count[stack[-1]] > 0:
                stack.pop()
            
            stack.append(letter)
            
            count[letter] -= 1
        
        return ''.join(stack)